In [1]:
from __future__ import print_function

import numpy as np
import time
import mxnet as mx


np.set_printoptions(suppress=True, precision=2)

import os
import shutil
from os import listdir

from os.path import join, isfile, isdir, split, splitext

from sklearn.model_selection import StratifiedKFold

from collections import defaultdict

from skimage.io import imread
import cv2 
from matplotlib import pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold


import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('dice_loss.log')
fh = logging.FileHandler('info.log')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
logger.info(123124)
logging.info(123)
logging.debug(123123)
batch_size = 3 #32
SIZE=192

import pickle

INFO:dice_loss.log:123124
INFO:root:123


loading dataset
finished loading dataset


In [ ]:
# loading model cell. you should change here path to model (search for /YOUR/PATH/TO/MODEL/WEIGHTS)


def build_unet_symbol_3d(coeff = 2):
    data = mx.symbol.Variable('data')
    conv1_1 = mx.sym.Convolution(data=data, kernel=(3, 3,3), num_filter=32 * coeff, pad=(1,1, 1), name='conv1_1')
    conv1_1bn = mx.sym.BatchNorm(data=conv1_1, name='conv1_1bn')
    act1_1 = mx.sym.Activation(data=conv1_1bn, act_type='relu', name='relu1_1')
    conv1_2 = mx.sym.Convolution(data=act1_1, kernel=(3, 3, 3), num_filter=32 * coeff, pad=(1,1, 1), name='conv1_2')
    conv1_2bn = mx.sym.BatchNorm(data=conv1_2, name='conv1_2bn')
    act1_2 = mx.sym.Activation(data=conv1_2bn, act_type='relu', name='relu1_2')
    pool1 = mx.symbol.Pooling(act1_2, pool_type="max", kernel=(2, 2, 2), stride=(2, 2, 2), name='pool1')
    
    conv2_1 = mx.sym.Convolution(data=pool1, kernel=(3, 3, 3), num_filter=64  * coeff, pad=(1,1,1), name='conv2_1')
    conv2_1bn = mx.sym.BatchNorm(data=conv2_1, name='conv2_1bn')
    act2_1 = mx.sym.Activation(data=conv2_1bn, act_type='relu', name='relu2_1')
    conv2_2 = mx.sym.Convolution(data=act2_1, kernel=(3, 3, 3), num_filter=64 * coeff, pad=(1,1, 1), name='conv2_2')
    conv2_2bn = mx.sym.BatchNorm(data=conv2_2, name='conv2_2bn')
    act2_2 = mx.sym.Activation(data=conv2_2bn, act_type='relu', name='relu2_2')
    pool2 = mx.symbol.Pooling(act2_2, pool_type="max", kernel=(2, 2, 2), stride=(2, 2, 2), name='pool2')
    
    
    conv3_1 = mx.sym.Convolution(data=pool2, kernel=(3,3,3), num_filter=128 * coeff, pad=(1,1,1), name='conv3_1')
    conv3_1bn = mx.sym.BatchNorm(data=conv3_1, name='conv3_1bn')
    act3_1 = mx.sym.Activation(data=conv3_1bn, act_type='relu', name='relu3_1')
    conv3_2 = mx.sym.Convolution(data=act3_1, kernel=(3,3,3), num_filter=128 * coeff, pad=(1,1,1), name='conv3_2')
    conv3_2bn = mx.sym.BatchNorm(data=conv3_2, name='conv3_2bn')
    act3_2 = mx.sym.Activation(data=conv3_2bn, act_type='relu', name='relu3_2')
    pool3 = mx.symbol.Pooling(act3_2, pool_type="max", kernel=(2, 2, 2), stride=(2, 2, 2), name='pool3')
    
    conv4_1 = mx.sym.Convolution(data=pool3, kernel=(3,3,3), num_filter=256 * coeff, pad=(1,1,1), name='conv4_1')
    conv4_1bn = mx.sym.BatchNorm(data=conv4_1, name='conv4_1bn')
    act4_1 = mx.sym.Activation(data=conv4_1bn, act_type='relu', name='relu4_1')
    conv4_2 = mx.sym.Convolution(data=act4_1, kernel=(3,3,3), num_filter=256 * coeff, pad=(1,1, 1), name='conv4_2')
    conv4_2bn = mx.sym.BatchNorm(data=conv4_2, name='conv4_2bn')
    act4_2 = mx.sym.Activation(data=conv4_2bn, act_type='relu', name='relu4_2')
    
    
    
    drop_conv5 = mx.sym.Dropout(data=act4_2, p=0.2, name='drop_conv5')
    
    up_conv6 = mx.symbol.Deconvolution(drop_conv5, kernel=(2, 2, 2), pad=(0, 0, 0),
                                    stride=(2, 2, 2), num_filter=256 * coeff, workspace=2048,
                                      name='up_conv6')
    concat6 = mx.symbol.Concat(up_conv6, act3_2, dim=1, name='concat6')
    conv7_1 = mx.sym.Convolution(data=concat6, kernel=(3,3,3), num_filter=128 * coeff, pad=(1,1,1), name='conv7_1')
    conv7_1bn = mx.sym.BatchNorm(data=conv7_1, name='conv7_1bn')
    act7_1 = mx.sym.Activation(data=conv7_1bn, act_type='relu', name='relu7_1')
    conv7_2 = mx.sym.Convolution(data=act7_1, kernel=(3,3,3), num_filter=128 * coeff, pad=(1,1,1), name='conv7_2')
    conv7_2bn = mx.sym.BatchNorm(data=conv7_2, name='conv7_2bn')
    act7_2 = mx.sym.Activation(data=conv7_2bn, act_type='relu', name='relu7_2')
    
    
    up_conv7 = mx.symbol.Deconvolution(act7_2, kernel=(2, 2, 2), pad=(0, 0, 0),
                                    stride=(2, 2, 2), num_filter=128 * coeff, workspace=2048,
                                      name='up_conv7')
    concat7 = mx.symbol.Concat(up_conv7, act2_2, dim=1, name='concat7')
    conv8_1 = mx.sym.Convolution(data=concat7, kernel=(3,3,3), num_filter=64 * coeff, pad=(1,1,1), name='conv8_1')
    conv8_1bn = mx.sym.BatchNorm(data=conv8_1, name='conv8_1bn')
    act8_1 = mx.sym.Activation(data=conv8_1bn, act_type='relu', name='relu8_1')
    conv8_2 = mx.sym.Convolution(data=act8_1, kernel=(3,3,3), num_filter=64 * coeff, pad=(1,1,1), name='conv8_2')
    conv8_2bn = mx.sym.BatchNorm(data=conv8_2, name='conv8_2bn')
    act8_2 = mx.sym.Activation(data=conv8_2bn, act_type='relu', name='relu8_2')
    
    up_conv8 = mx.symbol.Deconvolution(act8_2, kernel=(2, 2,2), pad=(0, 0,0),
                                    stride=(2, 2, 2), num_filter=128, workspace=2048,
                                      name='up_conv8')
    concat8 = mx.symbol.Concat(up_conv8, act1_2, dim=1, name='concat8')
    conv9_1 = mx.sym.Convolution(data=concat8, kernel=(3,3, 3), num_filter=32 * coeff, pad=(1,1,1), name='conv9_1')
    conv9_1bn = mx.sym.BatchNorm(data=conv9_1, name='conv9_1bn')
    act9_1 = mx.sym.Activation(data=conv9_1bn, act_type='relu', name='relu9_1')
    conv9_2 = mx.sym.Convolution(data=act9_1, kernel=(3,3,3), num_filter=32 * coeff, pad=(1,1,1), name='conv9_2')
    conv9_2bn = mx.sym.BatchNorm(data=conv9_2, name='conv9_2bn')
    act9_2 = mx.sym.Activation(data=conv9_2bn, act_type='relu', name='relu9_2')
    
    conv_output = mx.sym.Convolution(data=act9_2, kernel=(1,1,1), num_filter=1, name='conv_outut')
    output = mx.sym.Activation(data=conv_output, act_type='sigmoid', name='output')
    return output

output = build_unet_symbol_3d()
model = mx.mod.Module(symbol=output, context=mx.gpu(0))
print('binding data')
model.bind(data_shapes=[('data', (1, 1, 128, 192, 192))], for_training=False)
print('data is binded')
#ls /media/roman/b0529f6b-eab1-4917-bc61-a2111aca8d86/tclung3d/
params = mx.model.load_checkpoint('/YOUR/PATH/TO/MODEL/WEIGHTS/unet3d', 71)[1:]
model.set_params(*params)
print('done')

In [5]:
## Predicting cell. You should change here /YOUR/PATH/TO/EVALUATION/DATASET/system_extracted_no_gt/


def pixel2Mm(cntpt, slice_, dslice_):
    cntpt = cntpt[0]
    x = (cntpt[0] * dslice_[1]) + slice_[1];
    y = (cntpt[1] * dslice_[0]) + slice_[0];
    return x, y


def px_contour2mm(contour, meta):
    px_cnt = []
    for pt in contour:
        px_cnt.append(pixel2Mm(pt, meta[0], meta[1]))
    return px_cnt



def get_tumor_index(path2dir):
    """
    Extract tumor index to take contours
    """
    target_index = -1
    path2file = join(path2dir, 'structures.dat')
    with open(path2file) as f:
        data = f.readlines()
    assert(len(data) == 1), 'Structures.dat has different len for patient {}'.format(split(path2dir))
    data = np.array(['radiomics_gtv' in structure for structure in data[0].lower().split('|')])
    assert len(np.where(data ==True)[0]) == 1, "More than one tumor contours in structures.dat for patient {}".format(split(path2dir))
    target_index = np.where(data == True)[0][0] + 1
    return target_index
    

def extract_scan_meta(patientdir, scan_id):
    with open(join(patientdir, 'auxiliary', '{}.dat'.format(scan_id))) as f:
        for line in f:
            line = line.strip()
            parts = line.split(",")
            tag = parts[0]
            if tag == '(0020.0032)':
                x0, y0, z = float(parts[1]), float(parts[2]), float(parts[3])
            elif tag == '(0028.0010)':
                w = int(parts[1])
            elif tag == "(0028.0011)":
                h = int(parts[1])
            elif tag == "(0028.0030)":
                dx = float(parts[1])
                dy = float(parts[2])
            elif tag == "(0018.0050)":
                dz = float(parts[1])
    return ((y0, x0), (dy, dx))



def mm2Pixel(pt, meta):
    x = (pt[0] - meta[0][1]) / meta[1][1]
    y = (pt[1] - meta[0][0]) / meta[1][0]
    return (x, y)
    
def extract_tumor_contour_in_pixels(path2patient, scan_id, tumor_index, meta):
    """
    this function extracts tumor contours and translates it to pixels
    """
    #print('processing patient {} with scan {}'.format(split(path2patient), scan_id))
    cnts = []
    path2contour = join(path2patient, 'contours', '{}.{}.dat'.format(scan_id, tumor_index))
    if os.path.exists(path2contour):
        with open(path2contour) as f:
            for line in f:
                tumor_cnt = [float(el) for idx, el in enumerate(line.split(',')) if (idx + 1) % 3 != 0]
                assert len(tumor_cnt) % 2 == 0, "Wrong tumor cnt len"
                tumor_cnt = np.array(tumor_cnt).reshape(len(tumor_cnt) // 2, 2)
                for idx, pt in enumerate(tumor_cnt):
                    tumor_cnt[idx] = np.array(mm2Pixel(pt, meta))
                cnts.append(tumor_cnt)
    #print('Extracted {} contours for image {}'.format(len(cnts), join(split(path2patient)[1], scan_id)))
    return cnts

from mxnet import io

###### TODO ADD logic with inner contours
def evaluate_dataset(path2dir='/YOUR/PATH/TO/DATA/example_extracted/', thresh=None):
    """
    this function loads train dataset contours and images names
    """
    dataset = dict()
    solution = []
    for pat_idx, patiend in enumerate(listdir(path2dir)):
        if pat_idx % 100 == 0 and pat_idx != 0:
            print('done {} out of {}'.format(pat_idx, len(listdir(path2dir))))
            #break
        if not 'ANON_LUNG' in patiend:
            continue
        patiend = join(path2dir, patiend)
        scans_ids = sorted([int(k.split('.')[0]) for k in listdir(join(patiend, 'pngs')) if k.endswith('png')])
        dataset[split(patiend)[1]] = [[], []]
        for scanid in scans_ids:
            scan = '{}.png'.format(scanid)
            meta = extract_scan_meta(patiend, splitext(scan)[0])
            scan = cv2.resize(imread(join(path2dir, patiend, 'pngs', scan)), (192, 192))
            scan = (np.array(scan, dtype=np.float32) - 1000.)[np.newaxis]
            scan = scan[np.newaxis]
            dataset[split(patiend)[1]][0].append(scan)
            dataset[split(patiend)[1]][1].append(meta)
            
        """mask = cv2.resize(mask, (512, 512))
        _, contours, hierarchy = cv2.findContours(mask.astype(np.int32), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 20]

        contours_sizes = [cv2.contourArea(cnt) for cnt in contours]
        # transform to pixels
        contours = [px_contour2mm(cnt, meta) for cnt in contours if cv2.contourArea(cnt)]
        for cnt in contours:
            solution.append((split(patiend), splitext(scanid)[0], cnt))"""
        dataset[split(patiend)[1]][0] = np.rollaxis(np.array(dataset[split(patiend)[1]][0]), 0, 3)
        #print(dataset[split(patiend)[1]][0].shape)
    return dataset

from time import time
from itertools import chain
MAXES = []

solution = []
start = time()
dataset = evaluate_dataset('/YOUR/PATH/TO/EVALUATION/DATASET/system_extracted_no_gt/')
print("starting")
for pn in dataset:
    print("running pn", pn)
    mask = np.zeros(dataset[pn][0].shape[2:])
    current_patient_masks = []
    MAXES.append(dataset[pn][0].max())

    if dataset[pn][0].shape[2] <= 64:
        x = np.zeros((1, 1, 128, 192, 192), dtype=np.float32)
        x[0, 0, :dataset[pn][0].shape[2]] = dataset[pn][0][0,0]
        db = io.DataBatch([mx.nd.array(x)], [], 0)
        model.forward(db, is_train=False)
        preds = model.get_outputs()[0].asnumpy()[0, 0]
        for scanid in range(dataset[pn][0].shape[2]):
            #current_patient_masks.append(cv2.resize(preds[scanid], (512, 512)))
            current_patient_masks.append(preds[scanid])
    else:
        for zpos in range(64, dataset[pn][0].shape[2], 64):
            x = np.zeros((1, 1, 128, 192, 192), dtype=np.float32)
            zstart = max(0, zpos - 128//2)
            zend = min(dataset[pn][0].shape[2], zpos + 128//2)
            #x[ : (zend - zstart)] = cur_patient_data[0][zstart : zpos + SIZE//2]
            x[0,0, 0 : (zend - zstart), ...] = dataset[pn][0][0,0,zstart:zpos + 128//2]
            # make forward
            db = io.DataBatch([mx.nd.array(x)], [], 0)
            model.forward(db, is_train=False)
            preds = model.get_outputs()[0].asnumpy()[0, 0]
            # resize
            start_pos = 64 - 32
            if zpos == 64:
                start_pos = 0
            end_pos = 64 + 32
            if zpos + 64 >= dataset[pn][0].shape[2]:
                end_pos = (dataset[pn][0].shape[2] - zpos) + 64
            for copy_pos in range(start_pos, end_pos):
                current_patient_masks.append(cv2.resize((preds[copy_pos] > 0.5).astype(np.uint8), (512, 512)))
                _, contours, hierarchy = cv2.findContours(current_patient_masks[-1],
                                                          cv2.RETR_CCOMP,
                                                          cv2.CHAIN_APPROX_SIMPLE)
                contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 20]
                contours_sizes = [cv2.contourArea(cnt) for cnt in contours]
                # transform to pixels
                meta = dataset[pn][1][copy_pos]
                contours = [px_contour2mm(cnt, meta) for cnt in contours if cv2.contourArea(cnt)]
                for cnt in contours:
                    solution.append((pn, copy_pos + 1, cnt))
    assert len(current_patient_masks) == dataset[pn][0].shape[2], "something got wrong " + str(len(current_patient_masks)) + str(dataset[pn][0].shape)

print("evaluation took {} seconds".format(time() - start))


done 100 out of 136
starting
running pn ANON_LUNG_TC595


/usr/local/lib/python3.5/dist-packages/mxnet-0.10.1-py3.5.egg/mxnet/module/base_module.py:64: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


running pn ANON_LUNG_TC453
running pn ANON_LUNG_TC553
running pn ANON_LUNG_TC569
running pn ANON_LUNG_TC437
running pn ANON_LUNG_TC131
running pn ANON_LUNG_TC062
running pn ANON_LUNG_TC507
running pn ANON_LUNG_TC464
running pn ANON_LUNG_TC310
running pn ANON_LUNG_TC139
running pn ANON_LUNG_TC298
running pn ANON_LUNG_TC433
running pn ANON_LUNG_TC329
running pn ANON_LUNG_TC549
running pn ANON_LUNG_TC542
running pn ANON_LUNG_TC056
running pn ANON_LUNG_TC158
running pn ANON_LUNG_TC165
running pn ANON_LUNG_TC331
running pn ANON_LUNG_TC004
running pn ANON_LUNG_TC416
running pn ANON_LUNG_TC320
running pn ANON_LUNG_TC489
running pn ANON_LUNG_TC311
running pn ANON_LUNG_TC597
running pn ANON_LUNG_TC443
running pn ANON_LUNG_TC225
running pn ANON_LUNG_TC448
running pn ANON_LUNG_TC537
running pn ANON_LUNG_TC521
running pn ANON_LUNG_TC137
running pn ANON_LUNG_TC478
running pn ANON_LUNG_TC572
running pn ANON_LUNG_TC352
running pn ANON_LUNG_TC396
running pn ANON_LUNG_TC423
running pn ANON_LUNG_TC566
r

In [6]:
print(MAXES)
print(np.histogram(MAXES))

[3095.0, 1346.0, 1703.0, 3095.0, 3095.0, 3095.0, 2317.0, 2931.0, 2014.0, 2675.0, 14936.0, 1482.0, 3095.0, 3095.0, 3095.0, 3095.0, 1454.0, 3095.0, 3095.0, 3095.0, 3095.0, 5376.0, 3095.0, 3095.0, 1601.0, 3095.0, 3095.0, 10548.0, 3095.0, 3095.0, 3095.0, 2195.0, 3095.0, 3095.0, 3095.0, 24766.0, 11954.0, 3095.0, 2801.0, 4654.0, 3095.0, 3095.0, 3095.0, 2259.0, 3095.0, 3095.0, 3095.0, 2034.0, 3095.0, 17475.0, 3079.0, 2241.0, 3095.0, 1997.0, 3095.0, 2379.0, 2228.0, 1580.0, 3095.0, 3095.0, 3095.0, 3095.0, 4062.0, 3095.0, 3095.0, 3095.0, 3095.0, 3095.0, 3095.0, 1614.0, 3081.0, 2218.0, 1582.0, 2810.0, 2346.0, 3095.0, 1519.0, 3095.0, 3095.0, 3095.0, 3095.0, 1698.0, 3095.0, 5370.0, 2650.0, 2647.0, 3095.0, 3095.0, 1442.0, 3095.0, 1510.0, 3095.0, 3095.0, 3095.0, 3095.0, 3095.0, 3095.0, 1529.0, 3095.0, 3095.0, 3095.0, 2640.0, 3095.0, 3095.0, 3095.0, 2353.0, 11351.0, 3095.0, 3095.0, 17079.0, 3095.0, 1705.0, 3095.0, 3095.0, 2950.0, 3095.0, 2096.0, 10852.0, 3095.0, 1425.0, 4926.0, 3095.0, 3095.0, 3095.0,

In [8]:
# SAVING predictions

with open('system_extracted_no_gt.csv'.format(1), 'w') as f:
    for sol in solution:
        f.write(sol[0] + ',' + str(sol[1]) + ',' + ','.join([str(x) for x in chain(*sol[2])]) + '\n')

In [10]:
# running postprocessing script with bash from jupyter notebook
!python postprocess.py system_extracted_no_gt.csv system_seed_locations.csv system_extracted_no_gt_filtered.csv

Number of PIDs (contours):  124
Number of PIDs (seeds):  136

PID: ANON_LUNG_TC158
Seed Slices: [44]
Seed Padding: 7
Padded Seed Slices: [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]
Patient Contour Slices: [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 30, 31]
Contiguous Patient Contour Slices: [[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]]
Keeper Slices:  [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]

PID: ANON_LUNG_TC310
Seed Slices: [64]
Seed Padding: 7
Padded Seed Slices: [57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
Patient Contour Slices: [64, 65, 66, 67, 68, 69, 70, 71, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
Contiguous Patient Contour Slices: [[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]]
Keeper Slices:  [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]

PID: ANON_LUNG_TC311
Seed Slices: [7


PID: ANON_LUNG_TC503
Seed Slices: [77]
Seed Padding: 7
Padded Seed Slices: [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]
Patient Contour Slices: [54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]
Contiguous Patient Contour Slices: [[54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]]
Keeper Slices:  [54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]

PID: ANON_LUNG_TC375
Seed Slices: [85]
Seed Padding: 7
Padded Seed Slices: [78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
Patient Contour Slices: [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
Contiguous Patient Contour Slices: [[72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]]
Keeper Slices:  [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85


PID: ANON_LUNG_TC453
Seed Slices: [73, 77]
Seed Padding: 7
Padded Seed Slices: [66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]
Patient Contour Slices: [80, 76, 77, 78, 79]
Contiguous Patient Contour Slices: [[76, 77, 78, 79, 80]]
Keeper Slices:  [76, 77, 78, 79, 80]

PID: ANON_LUNG_TC067
Seed Slices: [101]
Seed Padding: 7
Padded Seed Slices: [94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
Patient Contour Slices: [99, 100, 101, 102, 103, 104, 105, 106, 107]
Contiguous Patient Contour Slices: [[99, 100, 101, 102, 103, 104, 105, 106, 107]]
Keeper Slices:  [99, 100, 101, 102, 103, 104, 105, 106, 107]

PID: ANON_LUNG_TC455
Seed Slices: [97, 83, 78]
Seed Padding: 7
Padded Seed Slices: [90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]
Patient Contour Slices: [33, 3

Keeper Slices:  [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]

PID: ANON_LUNG_TC597
Seed Slices: [61]
Seed Padding: 7
Padded Seed Slices: [54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]
Patient Contour Slices: [64, 65, 66, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]
Contiguous Patient Contour Slices: [[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]]
Keeper Slices:  [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]

PID: ANON_LUNG_TC329
Seed Slices: [43]
Seed Padding: 7
Padded Seed Slices: [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Patient Contour Slices: [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
Contiguous Patient Contour Slices: [[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]]
Keeper Slices:  [38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]

PID: ANON_LUNG_TC179
Seed Slices: [37]
Seed Padding: 7
Padded Seed Slices: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]



PID: ANON_LUNG_TC549
Seed Slices: [58]
Seed Padding: 7
Padded Seed Slices: [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
Patient Contour Slices: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]
Contiguous Patient Contour Slices: [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]]
Keeper Slices:  [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66]

PID: ANON_LUNG_TC546
Seed Slices: [90]
Seed Padding: 7
Padded Seed Slices: [83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]
Patient Contour Slices: [33, 34, 35, 36, 37, 38, 39, 40, 41, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
Contiguous Patient Contour Slices: [[33, 34, 35, 36, 37, 38, 39, 40, 41], [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]]
Keeper Slices:  [81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 